In [1]:
from pathlib import Path
import glob
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import copy

import open3d as o3d
import nibabel as nib
from skimage.measure import marching_cubes


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATA_ROOT = Path("C:/Users/aorhu/Masaüstü/samples")
TARGET_ROOT = "D:/decimated"
VERTICES = 25000
EXTENSION = ".ply"
LIMIT = 10000
COUNT = 0

In [3]:
def process(DATA_ROOT, TARGET_ROOT, VERTICES, EXTENSION, LIMIT, COUNT):

    for file in tqdm(os.listdir(str(DATA_ROOT))):
        _path = str(os.path.join(str(DATA_ROOT), file).replace('\\', '/')) +'/body_mask.nii.gz'
    
        _id = _path[_path[:_path.rfind("/")].rfind("/")+1:_path.rfind("/",0,)]
        
        if(os.path.exists(_path) and (LIMIT > COUNT) ):
            
            body_segment = nib.load(_path)
            body_segment_data = body_segment.get_fdata()

            verts, faces, _, __ = marching_cubes(body_segment_data, level=0, step_size=1)
            verts = verts/np.array(body_segment_data.shape) 

            mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                        triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

            decimated_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, VERTICES)
            
            if (len(decimated_mesh.triangles) == VERTICES):
                _target_path =  TARGET_ROOT + "/" + _id + EXTENSION
                print( _target_path)
                o3d.io.write_triangle_mesh( _target_path, decimated_mesh)
                print('\r' + str(COUNT) + "written", end='')
                COUNT += 1
        else:
            continue

In [4]:
process(DATA_ROOT, TARGET_ROOT, VERTICES, EXTENSION, LIMIT, COUNT)

 25%|█████████████████████                                                               | 1/4 [00:07<00:21,  7.14s/it]

D:/decimated/1000071.ply
0written

 50%|██████████████████████████████████████████                                          | 2/4 [00:13<00:13,  6.82s/it]

D:/decimated/1000125.ply
1written

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:20<00:06,  6.98s/it]

D:/decimated/1002371.ply
2written

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:30<00:00,  7.51s/it]


In [5]:
TARGET_ROOT = Path("D:/decimated")

In [6]:
for file in glob.glob(str(TARGET_ROOT / "*.ply")):
    _path = file.replace('\\', '/')
    print(_path)

D:/decimated/1000071.ply
D:/decimated/1000125.ply
D:/decimated/1002371.ply


In [7]:
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [8]:
source = o3d.io.read_point_cloud("D:/decimated/1000071.ply")
target = o3d.io.read_point_cloud("D:/decimated/1000125.ply")
threshold = 0.02

draw_registration_result(source, target)

In [9]:
source.estimate_normals()
target.estimate_normals()

In [10]:
threshold = 0.02

draw_registration_result(source, target)

In [12]:
trans_init = np.identity(4)

In [13]:
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [76]:
reg_p2l.transformation

array([[ 0.99908441, -0.01597169,  0.03968939, -0.02482255],
       [ 0.01370679,  0.99829736,  0.05669659, -0.04617666],
       [-0.04052735, -0.05610067,  0.99760225,  0.01434219],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
source_temp = copy.deepcopy(source)

In [16]:
source_temp.transform(reg_p2l.transformation)

PointCloud with 13324 points.

In [17]:
np.asarray(source.points)

array([[0.01060268, 0.35267857, 0.75619835],
       [0.01450893, 0.3452381 , 0.75826446],
       [0.01785714, 0.35119048, 0.7630854 ],
       ...,
       [0.99107143, 0.375     , 0.80440771],
       [0.99553571, 0.41666667, 0.88429752],
       [0.99553571, 0.3422619 , 0.88842975]])

In [18]:
np.asarray(source_temp.points)

array([[0.0101506 , 0.34892063, 0.74851216],
       [0.01425411, 0.3416635 , 0.75083243],
       [0.01769553, 0.34792497, 0.75517218],
       ...,
       [0.99127853, 0.38737643, 0.75561788],
       [0.99824402, 0.43356283, 0.83279768],
       [0.9995964 , 0.35951904, 0.84109416]])

In [19]:
o3d.visualization.draw_geometries([source_temp, source])

In [3]:
sample_pcd = o3d.io.read_point_cloud("D:/ADLM_Data/icp_target/4266049_5.ply")

In [4]:
o3d.visualization.draw_geometries([sample_pcd])

### reading as triangle mesh, converting to pcd, apply icp, convert to triangle mesh and write a triangle mesh

In [37]:
target_trm = o3d.io.read_triangle_mesh("D:/ADLM_Data/icp_target/4266049_25.ply")
target_trm.compute_vertex_normals()
target_pcd = o3d.geometry.PointCloud(points = target_trm.vertices)
target_pcd.estimate_normals()

In [38]:
o3d.visualization.draw_geometries([target_trm],mesh_show_back_face=True,mesh_show_wireframe=True)

In [17]:
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [39]:
source_trm = o3d.io.read_triangle_mesh("D:/ADLM_Data/decimated/1000071.ply")

In [40]:
source_trm

TriangleMesh with 13324 points and 25000 triangles.

#### convert to pcd

In [41]:
source_pcd = o3d.geometry.PointCloud(points = source_trm.vertices)
source_pcd.estimate_normals()

In [42]:
threshold = 0.02
trans_init = np.identity(4)

In [43]:
reg_p2l = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd, threshold, trans_init, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane())

In [44]:
reg_p2l.transformation

array([[ 0.99934494, -0.01791597,  0.0314438 , -0.01544623],
       [ 0.01796202,  0.99983797, -0.00118271, -0.00478638],
       [-0.03141752,  0.00174673,  0.99950482,  0.01857422],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [45]:
source_temp = copy.deepcopy(source_pcd)

In [46]:
source_temp.transform(reg_p2l.transformation)

PointCloud with 13324 points.

In [47]:
np.asarray(source_pcd.points)


array([[0.01060268, 0.35267857, 0.75619835],
       [0.01450893, 0.3452381 , 0.75826446],
       [0.01785714, 0.35119048, 0.7630854 ],
       ...,
       [0.99107143, 0.375     , 0.80440771],
       [0.99553571, 0.41666667, 0.88429752],
       [0.99553571, 0.3422619 , 0.88842975]])

In [48]:
np.asarray(source_temp.points)

array([[0.01260868, 0.34713113, 0.77468104],
       [0.01671064, 0.33975958, 0.77661041],
       [0.02010161, 0.34576543, 0.78133416],
       ...,
       [0.99355114, 0.38700312, 0.79210163],
       [0.99977804, 0.42864874, 0.8718844 ],
       [1.00124101, 0.35425114, 0.87588462]])

In [49]:
o3d.visualization.draw_geometries([source_temp, source_pcd])

In [50]:
new_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(source_temp.points)),
                                    triangles=o3d.utility.Vector3iVector(np.asarray(source_trm.triangles)))
new_mesh.compute_vertex_normals()

TriangleMesh with 13324 points and 25000 triangles.

In [51]:
original_mesh_paint = np.asarray([0,200,220])/255.0
registered_mesh_paint = np.asarray([230,200,110])/255.0
source_trm.paint_uniform_color(original_mesh_paint)
new_mesh.paint_uniform_color(registered_mesh_paint)

TriangleMesh with 13324 points and 25000 triangles.

In [52]:
source_trm.compute_vertex_normals()
new_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([source_trm,new_mesh],mesh_show_back_face=True,mesh_show_wireframe=True)